This notebook contain the steps to go from nedcow postgresql database to csv file.

In [1]:
import psycopg2
from sqlalchemy import create_engine
import pandas as pd
import numpy as np
%matplotlib inline

In [2]:
db_string = "postgresql:///nedcow-dev"

db = create_engine(db_string)

db = psycopg2.connect("dbname='nedcow-dev' host='localhost' password='mafia=612'")

cur = db.cursor()

cur.execute("Select * from results")

rows = cur.fetchall()

Data(rows) is returned as a list of tuples
The list of tuples contain
    index 0: Database index ID - type - int
    index 1: Store Name - type - str
    index 2: Product Information - type - Dict
    index 3: Amazon ASIN id - type - str
    index 4: Date - type - datetime object

In [3]:
rows[0]

(1,
 'target',
 {'amazon_sales_price': '75.98',
  'asin': 'B004QNL2CC',
  'image': 'http://ecx.images-amazon.com/images/I/51LTTrOVc9L._SL75_.jpg',
  'margin': '47.70',
  'pagelink': 'http://m.target.com/p/our-generation-20-morgan-horse-with-accessories/-/A-13407851',
  'profit': '16.69',
  'purchaseprice': '34.99',
  'salesrank': '17704',
  'title': 'Our Generation Poseable Morgan Horse For 18" Dolls',
  'upc': '062243242999'},
 'B004QNL2CC',
 datetime.datetime(2014, 6, 8, 19, 35, 45, 958895))

#### Make list of dictionaries to load into pandas Dataframe

In [4]:
# There has to be a more elegant way to do this
new_dict = {}
new_list = []
for row in rows:
    new_dict = row[2]
    new_dict['store'] = row[1]
    new_dict['date'] = row[4]
    new_list.append(new_dict)
 

In [14]:
df = pd.DataFrame(new_list)

In [15]:
df.shape

(263181, 18)

In [16]:
df = df[df['purchaseprice'] != '']

In [17]:
df.shape

(263177, 18)

#### Set columns to appropriate datatype
I am trying to convert string values that should be integer or floats

If a true string value exists within the values then I check the string size.

If the string size is greater than 10 characters then I set the value to 0.0
This done with the 'check_len' function.

In [18]:
df['amazon_sales_price'] = df.amazon_sales_price.replace(',','').astype(float)
df['profit'] = df.profit.replace(',','').astype(float)
df['margin'] = df.margin.replace(',','').astype(float)
df['salesrank']=df.salesrank.astype(int)

In [45]:
def check_len(x):
    if isinstance(x, str):
        x = x.strip().replace(',','')
        if len(x) > 20:
            x = '0'
            return x
        else:
            return x
    else:
        x = str(x)
        return x

In [49]:
df['purchaseprice'] = df.purchaseprice.apply(lambda x: check_len(x)).astype(float)

In [68]:
df.to_csv('goodstuff.csv.xz', compression='xz')

In [69]:
df.head()

,amazon_sales_price,asin,date,image,logo,margin,model,model#,pagelink,profit,purchaseprice,salesrank,shipping,sku,store,storepick,title,upc
0,75.98,B004QNL2CC,2014-06-08 19:35:45.958895,http://ecx.images-amazon.com/images/I/51LTTrOV...,NaN,47.70,NaN,NaN,http://m.target.com/p/our-generation-20-morgan...,16.69,34.99,17704,NaN,NaN,target,NaN,"Our Generation Poseable Morgan Horse For 18"" D...",062243242999
1,21.99,B007GQOCUK,2014-06-08 19:35:45.958895,http://ecx.images-amazon.com/images/I/61zYcBOF...,NaN,43.28,NaN,NaN,http://m.target.com/p/cra-z-art-my-look-color-...,4.54,10.49,147596,NaN,NaN,target,NaN,Cra Z Art My Look Color Your Own Messenger Bag,884920465091
2,36.95,B0073FV4CS,2014-06-08 19:35:45.958895,http://ecx.images-amazon.com/images/I/41uWeEHI...,NaN,63.68,NaN,NaN,http://m.target.com/p/beyblade-metal-masters-b...,10.82,16.99,205741,NaN,NaN,target,NaN,Beyblade Metal Fusion Battle Top Faceoff - Bur...,653569721697
3,29.95,B00AHVJ9V8,2014-06-08 19:35:45.958895,http://ecx.images-amazon.com/images/I/41TGh7cq...,NaN,40.29,NaN,NaN,http://m.target.com/p/furreal-friends-fancy-ki...,6.04,14.99,106122,NaN,NaN,target,NaN,FurReal Friends Dress Me Babies Fancy Kitty Pet,653569808534
4,69.95,B006O3F3KO,2014-06-08 19:35:45.958895,http://ecx.images-amazon.com/images/I/41Dkj6oZ...,NaN,83.81,NaN,NaN,http://m.target.com/p/our-generation-salon-cha...,22.62,26.99,235274,NaN,NaN,target,NaN,Our Generation Purple Salon Chair fits most 18...,062243243408
